# VASP Preprocessing

### By Adriana J. Ladera
### MIT, Center for Computational Science and Engineering

Notebook to create the VASP input files (POTCAR, KPOINTS, INCAR) given a POSCAR or .vasp file. This POSCAR or .vasp file is your structure! :D

- Source for all functions used in this notebook are in [VaspPreprocess](https://github.com/adrianaladera/SolidState-dont-Evaporate/blob/main/utils/VaspPreprocess.py)

- Full documentation of all VASP INCAR tages are found [here](https://www.vasp.at/wiki/index.php/Category:INCAR_tag)

NOTE: The energy cutoff INCAR tag (ENCUT), is defined by:

- $1.3 \times$ max([list of all ENMAX and ENMIN])

where ENMAX and ENMIN are defined in the POTCAR file. In most cases, this is sufficient to achieve an appropriate ENCUT.

In [2]:
import os
os.chdir("/Users/adrianaladera/Desktop/MIT/research/SolidState-dont-Evaporate/")

from utils import VaspPreprocess
from utils.packages import *

## Making relax/ directory based on the POSCAR in the main directory

VASP can relax a structure, i.e. minimize the total energy of the given system or minimize the forces per atom on the system using Density Functional Theory (DFT).

In this example, a relax/ directory is created and the VASP input files are created based on the path where the POSCAR or .vasp file is stored. 

The resulting tree will look like this:

    - original/path/to/POSCAR
    |
    |-- relax
        |- POSCAR
        |- POTCAR
        |- KPOINTS
        |- INCAR

The following INCAR tags for this example are set:
- `ISIF = 2` # relaxes the atomic positions but not cell shape or volume
- `NSW = 80` # maximum number of ionic steps
- `IBRION = 2` # relaxes ions/atoms using the conjugate gradient algorithm
- `IVDW = 11` #  DFT-D3 method of Grimme with zero-damping function

If you want to relax your structure, execute the following cell.

In [58]:
# path to where your POSCAR is stored; don't include the name of the POSCAR
root = "/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/fluorescent_1D/ligands"
					
mochas = [ "AgSe_py_RED"]

# mochas = 

for mocha in os.listdir(root):
    if os.path.isdir(f"{root}/{mocha}"):
        # print(mocha)

        path = f"{root}/{mocha}"

        filename = "sel_dyn.vasp"
        print(mocha, filename)

        # creating directories
        if not os.path.exists(f"{path}/relax"):
            os.mkdir(f"{path}/relax")
        if not os.path.exists(f"{path}/scf"):
            os.mkdir(f"{path}/scf")

        # renames filename to POSCAR for VASP input
        os.system(f"cp {path}/{filename} {path}/relax/POSCAR")
        print(f"{filename} copied to relax/POSCAR")

        structure = Structure.from_file(f"{path}/relax/POSCAR")
        VaspPreprocess.regular_kpoints(f"{path}/relax/", structure)
        VaspPreprocess.make_potcar(f"{path}/relax/")
        VaspPreprocess.relax_incar(f"{path}/relax/", isif=2, nsw=80, ibrion=2, isym=2, ivdw=11) # checks to see if POTCAR exists, then creates INCAR if True


galac_rtr_dehydrated_ligand sel_dyn.vasp
sel_dyn.vasp copied to relax/POSCAR
Regular KPOINTS written
POTCAR written
   ENMAX  =  101.968; ENMIN  =   76.476 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
Relaxation INCAR written
CuS-4MeO_ORANGE_ligand sel_dyn.vasp
sel_dyn.vasp copied to relax/POSCAR
Regular KPOINTS written
POTCAR written
   ENMAX  =  101.968; ENMIN  =   76.476 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
Relaxation INCAR written
Ag_2,6-dfSePh_1_YELLOW_ligand sel_dyn.vasp
sel_dyn.vasp copied to relax/POSCAR
Regular KPOINTS written
POTCAR written
   ENMAX  =  249.844; ENMIN  =  187.383 eV
   ENMAX  =  211.555; ENMIN  =  158.666 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
   ENMAX  =  216.285; ENMIN  =  162.214 eV
Rel

## Make a self-consistent field (SCF) folder either from a relaxed structure or an initial structure

You can get the energetics from a static calculation (SCF) after relaxing a structure, or if you want to get the energetics of a structure even without relaxing.

The resulting tree will look like this:

    - original/path/to/POSCAR
    |
    |-- relax
    |-- scf
        |- POSCAR
        |- POTCAR
        |- KPOINTS
        |- INCAR

IMPORTANT: must choose the correct file for which you want to run an SCF calculation. Be sure that if you want to run an SCF on a relaxed structure, you are copying the final CONTCAR from the relax/ folder into the SCF folder!

In [72]:
# is your structure relaxed? True if so, False if not (you just want a static calculation)
isrelaxed = True

root = "/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/fluorescent_1D/refined_structures_Dan_SD"
					
mochas = [ "galac_rtr", "Ag2-propane_v2"]

for mocha in mochas:
# for mocha in os.listdir(root):
    if os.path.isdir(f"{root}/{mocha}"):
        # print()
        print(mocha)
        path = f"{root}/{mocha}"
        # filename = "inorganic-with-H.vasp"

        if not os.path.exists(f"{path}/scf"):
            os.mkdir(f"{path}/scf")

        if isrelaxed:
            os.system(f"cp {path}/relax/CONTCAR {path}/scf/POSCAR")
            os.system(f"cp {path}/relax/run.slurm {path}/scf/")
            print(f"relax/CONTCAR copied to scf/POSCAR")
        else:
            os.system(f"cp {path}/{filename} {path}/scf/POSCAR")
            os.system(f"cp {path}/relax/run.slurm {path}/scf/")
            print(f"{filename} copied to scf/POSCAR")

        structure = Structure.from_file(f"{path}/scf/POSCAR")
        VaspPreprocess.regular_kpoints(f"{path}/scf/", structure)
        VaspPreprocess.make_potcar(f"{path}/scf/")
        VaspPreprocess.scf_incar(f"{path}/scf/") # checks to see if POTCAR exists, then creates INCAR if True

galac_rtr
relax/CONTCAR copied to scf/POSCAR
Regular KPOINTS written
POTCAR written
   ENMAX  =  400.000; ENMIN  =  300.000 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  249.844; ENMIN  =  187.383 eV
   ENMAX  =  258.689; ENMIN  =  194.016 eV
Self-consistent field (SCF) INCAR written
Ag2-propane_v2
relax/CONTCAR copied to scf/POSCAR
Regular KPOINTS written
POTCAR written
   ENMAX  =  249.844; ENMIN  =  187.383 eV
   ENMAX  =  258.689; ENMIN  =  194.016 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
Self-consistent field (SCF) INCAR written


## Electronic Structure Preprocessing

Electronic structure calculations include Density of States (DOS), and the band structure. 

Resulting tree will look like this:

    - original/path/to/POSCAR
    |
    |-- relax
    |-- scf
    |-- dos
        |- POSCAR
        |- POTCAR
        |- KPOINTS
        |- INCAR
        |- CHGCAR
    |-- band
        |- POSCAR
        |- POTCAR
        |- KPOINTS
        |- INCAR
        
The default functional is PBE in VASP. To use hybrid functionals (in this example, B3LYP), additional parameters have been added to the dos_incar() and band_incar() functions. B3LYP can be used simply by setting `ishybrid=True`.

In [74]:
# if true, then a hybrid functional (B3LYP) is used for the DOS and band calculations
ishybrid = False


root = "/Users/adrianaladera/Desktop/MIT/research/mochas/VASP_calculations/fluorescent_1D/inorganics"
					
mochas = [ "CuS_4PhCH3", "AgS_2-propane"]

for mocha in mochas:
# for mocha in os.listdir(root):
    if os.path.isdir(f"{root}/{mocha}"):
        # print()
        print(mocha)
        path = f"{root}/{mocha}"

        # potcar = f"{path}/scf/POTCAR"
        # structure = Structure.from_file(f"{path}/scf/POSCAR")
        structure = Structure.from_file(f"{path}/inorganic-with-H.vasp")

        VaspPreprocess.band_kpoints(structure, path) # creates the band/ and scf_band/ directories and primitive cells
        VaspPreprocess.make_potcar(f"{path}/scf_band/")
        VaspPreprocess.scf_incar(f"{path}/scf_band/")
        structure = Structure.from_file(f"{path}/scf_band/POSCAR")
        VaspPreprocess.regular_kpoints(f"{path}/scf_band/", structure)

        VaspPreprocess.band_incar(f"{path}/scf_band/POTCAR", f"{path}/band", ishybrid=ishybrid)
        VaspPreprocess.make_potcar(f"{path}/band/")

        # VaspPreprocess.dos_incar(f"{path}/scf/POTCAR", f"{path}/", ishybrid=ishybrid) # creates dos/ directory
        # if os.path.exists(f"{path}/scf/CHGCAR"):
        #     os.system(f"cp {path}/scf/POTCAR {path}/scf/POSCAR {path}/scf/CHGCAR {path}/scf/KPOINTS {path}/dos")
        #     print("POTCAR, POSCAR, KPOINTS, and CHGCAR copied from scf/ to dos/")
        # else:
        #     print("POTCAR, POSCAR, and KPOINTS copied from scf/ to dos/.\nWARNING: CHGCAR not copied because CHGCAR was not found.")

CuS_4PhCH3
Band KPOINTS written for default functional (PBE)
POTCAR written
   ENMAX  =  295.446; ENMIN  =  221.585 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  258.689; ENMIN  =  194.016 eV
Self-consistent field (SCF) INCAR written
Regular KPOINTS written
   ENMAX  =  295.446; ENMIN  =  221.585 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  258.689; ENMIN  =  194.016 eV
Band INCAR written
POTCAR written
AgS_2-propane
Band KPOINTS written for default functional (PBE)
POTCAR written
   ENMAX  =  249.844; ENMIN  =  187.383 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  258.689; ENMIN  =  194.016 eV
Self-consistent field (SCF) INCAR written
Regular KPOINTS written
   ENMAX  =  249.844; ENMIN  =  187.383 eV
   ENMAX  =  250.000; ENMIN  =  200.000 eV
   ENMAX  =  258.689; ENMIN  =  194.016 eV
Band INCAR written
POTCAR written


[101.968, 76.476, 250.0, 200.0, 400.0, 300.0, 400.0, 300.0]


520.0